In [1]:
import http.client
import json
import os
import pandas as pd
import time
import numpy as np

In [2]:
API_SOCCER = os.environ.get("API_SOCCER")

# Collecting Data

In [38]:
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': API_SOCCER }

In [39]:
# Eredivisie, Premier League, Bundesliga, Ligue 1, Serie A, Primera Divison
competitions = ['DED', 'PL', 'BL1', 'FL1', 'SA', 'PD']
seasons = [2018,2019,2020]

In [40]:
## Retreiving the data
for comp in competitions:
    for season in seasons:
        connection.request('GET', f"/v2/competitions/{comp}/matches?season={season}&status=FINISHED", None, headers )
        response = json.loads(connection.getresponse().read().decode())
        
        # Creating the empty DataFrame
        matches = pd.DataFrame()
        
        # Appending every match the the newly created DataFrame
        for match in response['matches']:
            matches = matches.append(match, ignore_index=True)
            
        ## Preprocessing
        # Correct type for ID and setting as index
        matches['id'] = matches['id'].astype('int32')
        matches = matches.set_index('id')
        matches.index = matches.index.astype('int64')
        # Remove odds
        matches = matches.drop(columns=['odds'])
        # Include winner as separate column
        matches['winner'] = [d.get('winner') for d in matches.score]
        # Include home team and away team as separate column
        matches['homeTeamName'] = [d.get('name') for d in matches.homeTeam]
        matches['awayTeamName'] = [d.get('name') for d in matches.awayTeam]
        
        ## Saving the data
        matches.to_csv(f"data/{comp}_{season}.csv")
        matches.to_pickle(f"data/{comp}_{season}.pickle")
        
        # Sleep 10 seconds, because we only have 10 calls per minute
        time.sleep(10)

## Combining datasets into a big one
We will first add a column indicating whether or not the date was pre/post corona, and then combine all datasets

In [1]:
# Eredivisie, Premier League, Bundesliga, Ligue 1, Serie A, Primera Divison
competitions = ['DED', 'PL', 'BL1', 'FL1', 'SA', 'PD']
seasons = [2018]

In [4]:
for comp in competitions:
    for season in seasons:
        df = pd.read_pickle(f"data/{comp}_{season}.pickle")
        df["corona"] = "pre"
        df.to_pickle(f"data/{comp}_{season}.pickle")

In [7]:
# Eredivisie, Premier League, Bundesliga, Ligue 1, Serie A, Primera Divison
competitions = ['DED', 'PL', 'BL1', 'FL1', 'SA', 'PD']
seasons = [2019]

In [10]:
# Dates manually looked up on worldfootball.net

for comp in competitions:
    for season in seasons:
        df = pd.read_pickle(f"data/{comp}_{season}.pickle")
        df["utcDate"] = pd.to_datetime(df["utcDate"]).dt.tz_localize(None)
        if comp == 'DED':
            df["corona"] = "pre"
            df.to_pickle(f"data/{comp}_{season}.pickle")
        elif comp == "PL":
            #1st of March, last game with fans
            df["corona"]= np.where(df["utcDate"] <= pd.Timestamp("2020-03-01").floor('D'),"pre","post")
            df.to_pickle(f"data/{comp}_{season}.pickle")
        elif comp == "BL1":
            #8th of March, last game with fans
            df["corona"]= np.where(df["utcDate"] <= pd.Timestamp("2020-03-08").floor('D'),"pre","post")
            df.to_pickle(f"data/{comp}_{season}.pickle")
        elif comp == "FL1":
            df["corona"] = "pre"
            df.to_pickle(f"data/{comp}_{season}.pickle")
        elif comp == "SA":
            #1st of March, last game with fans
            df["corona"]= np.where(df["utcDate"] <= pd.Timestamp("2020-03-01").floor('D'),"pre","post")
            df.to_pickle(f"data/{comp}_{season}.pickle")
        elif comp == "PD":
            #8th of March, last game with fans
            df["corona"]= np.where(df["utcDate"] <= pd.Timestamp("2020-03-08").floor('D'),"pre","post")            
            df.to_pickle(f"data/{comp}_{season}.pickle")
                

In [5]:
# Eredivisie, Premier League, Bundesliga, Ligue 1, Serie A, Primera Divison
competitions = ['DED', 'PL', 'BL1', 'FL1', 'SA', 'PD']
seasons = [2020]

In [6]:
for comp in competitions:
    for season in seasons:
        df = pd.read_pickle(f"data/{comp}_{season}.pickle")
        df["corona"] = "post"
        df.to_pickle(f"data/{comp}_{season}.pickle")

In [4]:
# Eredivisie, Premier League, Bundesliga, Ligue 1, Serie A, Primera Divison
competitions = ['DED', 'PL', 'BL1', 'FL1', 'SA', 'PD']
seasons = [2018,2019,2020]

In [5]:
final_df = pd.DataFrame()

for comp in competitions:
    for season in seasons:
        df = pd.read_pickle(f"data/{comp}_{season}.pickle")
        df["utcDate"] = pd.to_datetime(df["utcDate"]).dt.tz_localize(None)
        df["league"] = comp
        df["year"] = season
        final_df = final_df.append(df)
        
final_df.to_pickle("data/all_data.pickle")
final_df.to_csv("data/all_data.csv")